In [ ]:
# TEST_USCFCsv2OADCloud

## USCFCsv2OADCloud_Pure_SQLAlchemy

Carga del dataset desde local hacia Oracle Cloud utilizando pure SQLAlchemy.
En el ejemplo, se genera, en caso de no existir, una tabla con la siguiente estructura por defecto:

```
CREATE TABLE test_uscf90 (
	date_received CLOB, 
	product CLOB, 
	sub_product CLOB, 
	issue CLOB, 
	sub_issue CLOB, 
	consumer_complaint_narrative CLOB, 
	company_public_response CLOB, 
	company CLOB, 
	state CLOB, 
	zipcode CLOB, 
	tags CLOB, 
	consumer_consent_provided CLOB, 
	submitted_via CLOB, 
	date_sent_to_company CLOB, 
	company_response_to_consumer CLOB, 
	timely_response CLOB, 
	"consumer_disputed?" CLOB, 
	complaint_id NUMBER(19)
)
```

Una estructura manualmente definida como staging se adjunta a continuaci'on:

```
CREATE TABLE ADMIN.TEST_USCF03
(
  DATE_RECEIVED                 VARCHAR2(4000 CHAR),
  PRODUCT                       VARCHAR2(4000 CHAR),
  SUB_PRODUCT                   VARCHAR2(4000 CHAR),
  ISSUE                         VARCHAR2(4000 CHAR),
  SUB_ISSUE                     VARCHAR2(4000 CHAR),
  CONSUMER_COMPLAINT_NARRATIVE  VARCHAR2(10000 CHAR), -- Ver parametro max_string_size
  COMPANY_PUBLIC_RESPONSE       VARCHAR2(4000 CHAR),
  COMPANY                       VARCHAR2(4000 CHAR),
  STATE                         VARCHAR2(4000 CHAR),
  ZIPCODE                       VARCHAR2(4000 CHAR),
  TAGS                          VARCHAR2(4000 CHAR),
  CONSUMER_CONSENT_PROVIDED     VARCHAR2(4000 CHAR),
  SUBMITTED_VIA                 VARCHAR2(4000 CHAR),
  DATE_SENT_TO_COMPANY          VARCHAR2(4000 CHAR),
  COMPANY_RESPONSE_TO_CONSUMER  VARCHAR2(4000 CHAR),
  TIMELY_RESPONSE               VARCHAR2(4000 CHAR),
  IS_CONSUMER_DISPUTED          VARCHAR2(4000 CHAR),
  COMPLAINT_ID                  VARCHAR2(4000 CHAR)
)
```

Una estructura con los tipos pre-definidos y normalizados (requiere normalizar previamente).

_Notas_ sobre la normalizaci'on:

COMPLAINT_ID no es requerido a menos que el campo de CONSUMER_COMPLAINT_NARRATIVE se incluya fuera en otra tabla espec'ifica.


```
CREATE TABLE ADMIN.TEST_USCF03
(
  DATE_RECEIVED                 DATE,
  PRODUCT                       NUMBER,
  SUB_PRODUCT                   NUMBER,
  ISSUE                         NUMBER,
  SUB_ISSUE                     NUMBER,
  CONSUMER_COMPLAINT_NARRATIVE  VARCHAR2(10000 CHAR),
  COMPANY_PUBLIC_RESPONSE       NUMBER,
  COMPANY                       NUMBER,
  STATE                         NUMBER,
  ZIPCODE                       NUMBER,
  TAGS                          NUMBER,
  CONSUMER_CONSENT_PROVIDED     NUMBER,
  SUBMITTED_VIA                 NUMBER,
  DATE_SENT_TO_COMPANY          DATE,
  COMPANY_RESPONSE_TO_CONSUMER  NUMBER,
  TIMELY_RESPONSE               NUMBER,
  IS_CONSUMER_DISPUTED          NUMBER,
  COMPLAINT_ID                  NUMBER
)
```


In [ ]:
import time
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from datetime import datetime

import getpass

import matplotlib.pyplot as plt

# TODO Agregar soporte para modulo logging

# -- Datetime format
STRFMT = "%m/%d/%Y, %H:%M:%S.%f"

# -- Data Path con el dataset a cargar 
DATA_IN_PATH=r"C:\data\33_793516_bundle_archive\consumer_complaints.csv"

# -- Definicion del PATH donde hemos instalado Oracle Instant Client
# -- Inicializaremmos de forma explicita
ORA_INST_CLIENT_PATH = r"C:\instantclient_19_8"

# -- 
ORA_INST_CLIENT_TNSC = r"<TNS_ENTRADA>"

# -- Solicitamos usuario y password, no recomiendo hardcode de los
# -- mismos en el codigo.
oc_user = input('AD user')
oc_pass = getpass.getpass('AD password')

# -- Oracle Table: TEST_USCF90
tbl_name = 'test_uscf90'

# -- Nro de filas por cada loop  
nrows = 10

# -- Nro de loops a realizar
n_loops = 10

# -- Insert Mode
if_exists_mode = 'append'

engine = create_engine(
    "oracle+cx_oracle://{0}:{1}@{2}".format(oc_user, oc_pass, ORA_INST_CLIENT_TNSC), echo = True)

df = pd.read_csv(DATA_IN_PATH)

df.info()

# -- Ver toda la informacion de las columnas
pd.set_option('display.max_columns', None)

# -- Activa o no la creacion de un indice en BD
b_index = False

x_plot = []
y_plot = []

# TODO Tambien podemos utilizar %timeit -n 10
for i in range(n_loops): 
    print('{0} [INFO] Inicio del loop {1}'.format(datetime.now().strftime(STRFMT), i))
    start_t = time.time() 
    df[:nrows].to_sql(tbl_name, index=b_index, con=engine, if_exists=if_exists_mode )
    end_t = time.time()
    print('{0} [INFO] Tiempo para el loop {1} - {2} filas: {3}'.format(datetime.now().strftime(STRFMT), \
        i, nrows, round(end_t-start_t, 1)))
    print('{0} [INFO] Tiempo total estimado para el loop {1} - {2} filas: {3}'.format(datetime.now().strftime(STRFMT), \
        i, df.shape[0],df.shape[0] * (end_t-start_t) / nrows))

    # -- Para plotear los tiempos finales
    x_plot.append(i)
    y_plot.append((end_t-start_t)/nrows)

print('{0} [INFO] Fin del Test'.format(datetime.now().strftime(STRFMT)))

y_pos = np.arange(len(x_plot))

plt.bar(y_pos, y_plot, align='center', alpha=0.5)
plt.xticks(y_pos, x_plot)
plt.xticks(rotation=0)
plt.tick_params(
    axis='x',          
    which='both',      
    bottom=True,      
    top=False,         
    labelbottom=True) 

plt.ylabel('Tiempo(sec) x Fila')
plt.title('{0} loops -> {1} filas/loop'.format(n_loops, nrows))
plt.show()
  